# 0. Station Information

* Station name: Trout Beck Calgarth Gauging Station E 201704
* Data link: https://environment.data.gov.uk/hydrology/id/stations/E06072A

In [138]:
# Import the relevant libraries
import os
import pandas as pd
from pathlib import Path

In [139]:
# Create directories for Python scripts
os.makedirs("data_preparation", exist_ok=True) # create data preparation directory

# 1. Download the data

In [140]:
%%writefile load_raw_data.py
"""
Python script to load the data.

Usage: Takes in a ZIP archive from a target URL and extracts the contents to a target directory.
"""

# Import the required libraries
import os
import zipfile
from pathlib import Path
import requests
import argparse

# Create an argument parser
parser = argparse.ArgumentParser()
# Obtain a data URL argument
parser.add_argument("--url",
                    type=str,
                    help="URL for the data.")
# Obtain the argument from the parser
args = parser.parse_args()

# Establish the path to the raw data directory
data_path = Path("data/")
raw_data_path = data_path/"raw_data"

# Create the raw data directory if it does not already exist
if raw_data_path.is_dir():
  print(f"[INFO] {raw_data_path} directory already exists.")
else:
  raw_data_path.mkdir(parents=True, exist_ok=True)
  print(f"[INFO] {raw_data_path} directory has been created.")

# Download the ZIP archive containing the raw water quality parameter data
with open(raw_data_path/"raw_water_quality_parameter_data.zip", "wb") as f:
  response = requests.get(args.url)
  f.write(response.content)

# Extract the contents of the downloaded ZIP archive
try:
  with zipfile.ZipFile(raw_data_path/"raw_water_quality_parameter_data.zip", "r") as zip_ref:
    zip_ref.extractall(raw_data_path)
except zipfile.BadZipFile:
  print(f"[ERROR] the file {zip_ref} is not a valid ZIP file.")
  exit(1)

# Remove the ZIP archive
os.remove(raw_data_path/"raw_water_quality_parameter_data.zip")

Overwriting load_raw_data.py


In [141]:
# Load the data using the `load_raw_data.py` script
!python load_raw_data.py --url https://github.com/VinceMoran/EA_Water_Quality_Time_Series_Prediction/raw/main/data/raw_data/raw_water_quality_parameter_data.zip

[INFO] data/raw_data directory already exists.


# 2. Load the data into pandas DataFrames

In [142]:
# Assign the filepaths for the raw parameter data
ammonium_path = Path("/content/data/raw_data/TROUT-BECK_CALGARTH-GAUGING-STATION_E_201704-AMMONIUM-15min-Measured.csv")
chlorophyll_path = Path("/content/data/raw_data/TROUT-BECK_CALGARTH-GAUGING-STATION_E_201704-CHLOROPHYLL-15min-Measured.csv")
conductivity_path = Path("/content/data/raw_data/TROUT-BECK_CALGARTH-GAUGING-STATION_E_201704-CONDUCTIVITY-15min-Measured.csv")
oxygen_path = Path("/content/data/raw_data/TROUT-BECK_CALGARTH-GAUGING-STATION_E_201704-DISSOLVED OXYGEN-15min-Measured.csv")
ph_path = Path("/content/data/raw_data/TROUT-BECK_CALGARTH-GAUGING-STATION_E_201704-PH-15min-Measured.csv")
temp_path = Path("/content/data/raw_data/TROUT-BECK_CALGARTH-GAUGING-STATION_E_201704-TEMPERATURE-15min-Measured.csv")
turbidity_path = Path("/content/data/raw_data/TROUT-BECK_CALGARTH-GAUGING-STATION_E_201704-TURBIDITY-15min-Measured.csv")

In [143]:
# Load the data into pandas DataFrames
ammonium_df = pd.read_csv(ammonium_path)
chlorophyll_df = pd.read_csv(chlorophyll_path)
conductivity_df = pd.read_csv(conductivity_path)
oxygen_df = pd.read_csv(oxygen_path)
ph_df = pd.read_csv(ph_path)
temp_df = pd.read_csv(temp_path)
turbidity_df = pd.read_csv(turbidity_path)

# 3. Investigate the raw data DataFrames

In [144]:
# Define a function for computing the unqiue value counts of variables
def unique_value_counts(dataframe: pd.DataFrame,
                        dataframe_name: str):
  """
  Takes in a pandas DataFrame and prints the unique value counts for each
  variable.

  Args:
    dataframe (pd.DataFrame): A pandas DataFrame to be investigated.
    dataframe_name (str): The name of the DataFrame to be printed for clarity.

  Returns:
    None.
  """
  # Print the output header
  print(f"DISTINCT VALUE COUNTS FOR {dataframe_name}:")
  print("{:16} {:^25}".format("Variable", "Number of Distinct Value Counts")) # print a header row for the table

  # Loop through the DataFrame columns and print a row for each variable
  for variable in dataframe.columns:
    print("{:16} {:^25}".format(variable, dataframe[variable].nunique()))  # Use nunique() to get unique value count
    print()

In [145]:
# Compute the unique value distributions for each DataFrame
unique_value_counts(ammonium_df, "AMMONIUM DATAFRAME")
print("\n----------------------------------------")
unique_value_counts(chlorophyll_df, "CHLOROPHYLL DATAFRAME")
print("\n----------------------------------------")
unique_value_counts(conductivity_df, "CONDUCTIVITY DATAFRAME")
print("\n----------------------------------------")
unique_value_counts(oxygen_df, "OXYGEN DATAFRAME")
print("\n----------------------------------------")
unique_value_counts(ph_df, "PH DATAFRAME")
print("\n----------------------------------------")
unique_value_counts(temp_df, "TEMPERATURE DATAFRAME")
print("\n----------------------------------------")
unique_value_counts(turbidity_df, "TURBIDITY DATAFRAME")

DISTINCT VALUE COUNTS FOR AMMONIUM DATAFRAME:
Variable         Number of Distinct Value Counts
measure                      1            

dateTime                  100000          

date                       2558           

value                      1794           

completeness                 0            

quality                      1            

qcode                        0            


----------------------------------------
DISTINCT VALUE COUNTS FOR CHLOROPHYLL DATAFRAME:
Variable         Number of Distinct Value Counts
measure                      1            

dateTime                  100000          

date                       2558           

value                      2211           

completeness                 0            

quality                      1            

qcode                        0            


----------------------------------------
DISTINCT VALUE COUNTS FOR CONDUCTIVITY DATAFRAME:
Variable         Number of Distinct Value Counts
measure 

In [146]:
# Define a function for computing datatypes and checking missing values
def datatypes_and_missing_values(dataframe: pd.DataFrame,
                                 dataframe_name: str):
    """
    Takes in a pandas DataFrame and prints the datatype and number of missing
    values for each variable.

    Args:
      dataframe (pd.DataFrame): A pandas DataFrame to be investigated.
      dataframe_name (str): The name of the DataFrame to be printed for clarity.

    Returns:
      None.
    """
    # Print the output header
    print(f"DATATYPES AND MISSING VALUES FOR {dataframe_name}")
    print("{v: <18} {dt: <14} {n: ^24}".format(v="Variable", dt="Data Type", n="Number of Missing Values")) # Print a header row for the table.

    # Loop through the DataFrame columns and print a row for each variable
    for variable in dataframe.columns:
        print("{v: <18}, {dt: <14}, {n: ^24}".format(v=variable, dt=str(dataframe[variable].dtype), n=dataframe[variable].isnull().sum()))

In [147]:
# Determine the datatype and number of missing values for each variable
datatypes_and_missing_values(ammonium_df, "AMMONIUM DATAFRAME")
print("\n----------------------------------------")
datatypes_and_missing_values(chlorophyll_df, "CHLOROPHYLL DATAFRAME")
print("\n----------------------------------------")
datatypes_and_missing_values(conductivity_df, "CONDUCTIVITY DATAFRAME")
print("\n----------------------------------------")
datatypes_and_missing_values(oxygen_df, "OXYGEN DATAFRAME")
print("\n----------------------------------------")
datatypes_and_missing_values(ph_df, "PH DATAFRAME")
print("\n----------------------------------------")
datatypes_and_missing_values(temp_df, "TEMPERATURE DATAFRAME")
print("\n----------------------------------------")
datatypes_and_missing_values(turbidity_df, "TURBIDITY DATAFRAME")

DATATYPES AND MISSING VALUES FOR AMMONIUM DATAFRAME
Variable           Data Type      Number of Missing Values
measure           , object        ,            0            
dateTime          , object        ,            0            
date              , object        ,            0            
value             , float64       ,            0            
completeness      , float64       ,          100000         
quality           , object        ,            0            
qcode             , float64       ,          100000         

----------------------------------------
DATATYPES AND MISSING VALUES FOR CHLOROPHYLL DATAFRAME
Variable           Data Type      Number of Missing Values
measure           , object        ,            0            
dateTime          , object        ,            0            
date              , object        ,            0            
value             , float64       ,            0            
completeness      , float64       ,          100000         
q

# 4. Merge the DataFrames into a single DataFrame
* dateTime column was selected as the common index for time series data.

In [148]:
# Rename columns prior to merging to avoid clashes
dataframes_list = [ammonium_df, chlorophyll_df, conductivity_df, oxygen_df, ph_df, temp_df, turbidity_df] # store DataFrames in a list to condense operations
new_column_titles = ['ammonium', 'chlorophyll', 'conductivity', 'dissolved_oxygen', 'ph', 'temperature', 'turbidity'] # store new names for DataFrame columns to condense operations

# Loop through DataFrame list and change column names
for i in range(len(dataframes_list)):
  dataframes_list[i] = dataframes_list[i].rename(columns={'value': new_column_titles[i]})

In [149]:
# Chain merge the DataFrames into a single DataFrame containing dateTime and parameter measurements
for i in range(len(dataframes_list)-1):
  if i == 0:
    df_parameters = pd.merge(dataframes_list[i][['dateTime', new_column_titles[i]]],
                             dataframes_list[i+1][['dateTime', new_column_titles[i+1]]],
                             on='dateTime')
  else:
    df_parameters = pd.merge(df_parameters,
                             dataframes_list[i+1][['dateTime', new_column_titles[i+1]]],
                             on='dateTime')
df_parameters

,dateTime,ammonium,chlorophyll,conductivity,dissolved_oxygen,ph,temperature,turbidity
0,1999-11-30T00:07:00,0.09,0.88,100.81,10.38,8.08,15.602,4.26
1,1999-11-30T00:50:06,0.01,0.33,84.97,12.98,7.62,5.606,1.91
2,2017-03-29T09:33:44,112.09,0.54,1.03,9.39,7.48,18.026,15.80
3,2017-03-29T09:36:34,9.34,0.82,1.03,9.45,7.97,18.074,53.26
4,2017-03-29T09:41:50,6.92,1.00,0.98,9.42,8.14,18.193,32.12
...,...,...,...,...,...,...,...,...
99995,2024-07-02T06:31:12,0.01,1.14,72.20,10.66,7.38,11.983,0.53
99996,2024-07-02T07:01:12,0.01,1.14,72.22,10.75,7.39,11.998,0.49
99997,2024-07-02T07:31:12,0.01,1.26,72.24,10.81,7.43,12.014,0.49
99998,2024-07-02T08:01:12,0.01,1.27,72.22,10.75,7.40,12.046,0.51


# 5. Investigate and pre-process the merged DataFrame

In [150]:
# Compute the unique value distributions for the merged DataFrame
unique_value_counts(df_parameters, "PARAMETER DATAFRAME")
print("\n----------------------------------------")

# Determine the datatype and number of missing values for each variable
datatypes_and_missing_values(df_parameters, "PARAMETER DATAFRAME")

DISTINCT VALUE COUNTS FOR PARAMETER DATAFRAME:
Variable         Number of Distinct Value Counts
dateTime                  100000          

ammonium                   1794           

chlorophyll                2211           

conductivity               10330          

dissolved_oxygen            794           

ph                          582           

temperature                16236          

turbidity                  6206           


----------------------------------------
DATATYPES AND MISSING VALUES FOR PARAMETER DATAFRAME
Variable           Data Type      Number of Missing Values
dateTime          , object        ,            0            
ammonium          , float64       ,            0            
chlorophyll       , float64       ,            0            
conductivity      , float64       ,            0            
dissolved_oxygen  , float64       ,            0            
ph                , float64       ,            0            
temperature       , float64     

In [151]:
# Convert the dateTime column to the datetime64 data type
df_parameters['dateTime'] = pd.to_datetime(df_parameters['dateTime'])

In [152]:
# Determine the datatype and number of missing values for each variable
datatypes_and_missing_values(df_parameters, "PARAMETER DATAFRAME")

DATATYPES AND MISSING VALUES FOR PARAMETER DATAFRAME
Variable           Data Type      Number of Missing Values
dateTime          , datetime64[ns],            0            
ammonium          , float64       ,            0            
chlorophyll       , float64       ,            0            
conductivity      , float64       ,            0            
dissolved_oxygen  , float64       ,            0            
ph                , float64       ,            0            
temperature       , float64       ,            0            
turbidity         , float64       ,            0            


# 6. Save the merged DataFrame to a csv file

In [153]:
# Specify the file path for saving the DataFrame as a csv file
df_parameters_path = Path("/content/data/df_parameters.csv")

# Save the DataFrame to a csv file
df_parameters.to_csv(df_parameters_path, index=False)